In [70]:
import keras
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,LSTM,Input,Bidirectional
from sklearn.model_selection import cross_val_score 
from nltk.tokenize import word_tokenize,sent_tokenize
import nltk

In [71]:
from keras.preprocessing.text import Tokenizer
from gensim.models.fasttext import FastText
import matplotlib.pyplot as plt
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk import WordPunctTokenizer

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Saswat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Saswat\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Saswat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [72]:
from nltk.translate.bleu_score import sentence_bleu
def bleu_score(reference,candidate):
  reference=[word_tokenize(reference)] 
  candidate=word_tokenize(candidate)
  score = sentence_bleu(reference, candidate)
  return score

In [73]:
def isint(word):
  try:
    word=int(word) 
  except:
    word=word 
  if type(word)==int and int(word)==float(word):
    return 1 
  else:
    return 0
def isNan(word):
    try:
        word=int(word) 
    except:
        word=0 
    return word
def isfloat(word):
  try:
    word=float(word) 
  except:
    word=word 
  if type(word)==int:
    return 1 
  else:
    return 0 

def get_pos(arr):
  ans=[]
  for i in range(len(arr)):
    if arr[i]>0.08:
      ans.append(i) 
  return ans


In [74]:
clean_mp={'6gb': '6 gb', '34gb': '34 gb', '128gb': '128 gb', '3gb': '3 gb','16-megapixel':'16 mp','with15mp': 'with 15 mp', '10mp': '10 mp', '8mp': '8 mp', 'mp-primary': 'mp for primary', 'mp-secondary': 'mp for secondary', '13mp': '13 mp', '16mp': '16 mp', '23mp': '23 mp', '20mp': '20 mp', '12mp': '12 mp','4gb':'4 gb','dualsim':'dual sim','feauture':'feature','gread':'great','..':'.','resposnse':'response','dscount':'discount','ofcourse':'of course','non-smartphone':'non smartphone','freindly':'friendly','perfec':'perfect','rs9999':'rs 9999','asssist':'assist','9000/-':'9000','speciific':'specific','phonebecause':'phone because','188gms':'188 gms','168gms':'168 gms','possesor':'processor','ramand':'ram and','youngeters':'youngsters','multi-':'multi','than32':'than 32','ssure':'sure','5megapixel':'5 mp','addtional':'additional','widely-adopted':'widely adopted','speciifc':'specific','avilable':'available','appliactions':'appliacations','xiomi':'xiaomi','multisim':'multi sim','applehas':'apple has','buawei':'huawei','specificctions':'specifications','non-smartphones':'non smartphones','spcific':'specific','itune':'itunes','brandyou':'brand you','goood':'good','awsome':'awesome','dual-primary':'dual primary','selfie':'photo','multitaking':'multitasking','widely-adopted':'widely adopted','gioned':'gionee','coolp':'coolpad','adbrand':'brand','advanatge':'advantage','ecpecially':'especially','movies/netflix':'movies and netflix','eqipped':'equipped','samsungmobile':'samsung mobile','samusng':'samsung','260euro':'260 euro', '750euro':'750 euro', '250euro':'250 euro', '550euro':'550 euro', '750euro':'750 euro'}
val_mp={'good': 'good', 'black': 'black', 'android': 'and', 'y': 'yes', 'white': 'white', 'grey': 'grey', 'less': 'less', 'high': 'high', '15k': '15k', '10k': '10k', 'latest': 'latest', 'non': 'none', 'no': 'no', 'vivo': 'vivo', 'i': 'i', 'gold': 'gold', '15+': '15', 'windows': 'windows', '15k+': '15k', 'nokia': 'nokia', 'e': 'e', '16k+': '16k', '20k': '20k', '2019': '2019', '2k': '2k', '6gb': '6gb', '15': '15', 'jio': 'jio', 'f': 'f', 'blue': 'blue', '4+': '4', 'silver': 'silver', 'big': 'big', 'light': 'light', 'htc': 'htc', 'paradise': 'paradise', 'lenovo': 'lenovo', 'yoga tab 3': 'yoga tab 3', 'yes': 'yes', '400': '400', ' light': 'light', '64gb': '64gb', 'd': 'dual', 't': 't-mobile', 'non black': 'non-black', 'small': 'small', 'non and': 'ios', 'samsung': 'samsung', 'samsung ': 'samsung', 'expensive': 'expensive', '128': '128', 'non silver': 'non-silver', '16+': '16', '2016': '2016', '500': '500', '2017': '2017', '128 gb': '128gb', '800+': '800', 'apple': 'apple', '700': '700', '20': '20', 'and': 'and', 'black&blue': 'black', '650': '650', '64+': '64', 'may': 'may', '4000+': '4000', 'pink': 'pink', 'good  ': 'good', '400+': '400', '64': '64', 'non gunmetal': 'non-gunmetal', 'sony': 'sony', 'xperia xa ultra': 'xperia xa ultra', 'non oppo': 'vivo', 'lg': 'lg', 'acer': 'acer', 'iconia tab a700 ': 'iconia tab a700', 'non grey': 'non-grey', 'oppo': 'oppo', '350': '350', 'non crown gold': 'non-gold', 'pearl white\n': 'white', 'recent': 'recent', '300': '300', '20+': '20', '32+': '32', 'blue||gold': 'blue||gold', '8000+': '8000', 'motorola': 'motorola', '150': '150', '12+': '12', '6': '6', '3': '3', '32': '32', 'galaxy note7': 'galaxy note7', '3000+': '3000', 'null': 'null', '13+': '13', '250': '250', 'aquamarine green': 'green', 's': 's', '200': '200', 'pearl white': 'white', 'huawei': 'huawei', 'u8150 ideos': 'u8150 ideos', 'blackberry': 'blackberry', '6000+': '6000', 'good      ': 'good', 'm': 'm', '16': '16', 'onyx black': 'black', 'green': 'green', 'heavy': 'heavy', 'bla': 'bla', ' 300': '300', '220': '220', 'sym': 'sym', 'iconia tab a501\n': 'iconia tab a501', ' non and': 'ios', 'mic': 'mic', 'non dual': 'non-dual', 'titanium grey': 'grey', 'xiaomi': 'xiaomi', '800': '800', 'ios': 'ios', '64 gb': '64gb', '16 ': '16', '': '', 'chrome': 'chrome', 'gray': 'gray', 'non blue': 'non-blue', 'sony ericsson': 'ericsson', 'xperia arc s': 'xperia', '3500+': '3500', 'good     ': 'good', '10+': '10', 'asus': 'asus', '450': '450', '340': '340', 'leeco\n': 'leeco', '160': '160', 'medium': 'medium', 'good        ': 'good', '2014': '2014', 'non gold': 'non-gold', '600': '600', '4': '4', '2015': '2015', '270': '270', '4.65 hd': 'hd', ' 250': ' 250', 'average': 'average', 'good       ': 'good', 'dual': 'dual', ' and': 'and', 'non vga': 'non-vga', 'orange': 'orange', 'xiamo': 'xiaomi', '1800+': '1800', 'gionee': 'gionee', '5': '5', '140': '140', 'non coolpad': 'non-coolpad', 'alcatel': 'alcatel', 'allview': 'allview', 'zte': 'zte', 'xiaomo ': 'xiaomi', '500+': '500', 'moderate': 'moderate', 'coolpad': 'coolpad', 'gigabyte': 'gigabyte', '2+': '2', 'tiz': 'tiz', 'micromax': 'micromax', '13': '13', 'non white': 'non-white', '280': '280', '3000': '3000', 'vodafone': 'vodafone', 'charcoal black\n': 'black', 'oneplus': 'oneplus', 'vertu': 'vertu', '8': '8', '260': '260', '8+': '8', ' bla': ' bla', '3+': '3', 'meizu': 'meizu', 'archos': 'archos', 'google': 'google', '4500+': '4500', 'hp': 'hp'} 
intent_mp={'Phn_SelReq': 'selling', 'Booking_Req': 'booking'}
u_action_mp={'null':'null','Phn_SelReq': 'phone', 'Rec_Slt': 'rec_slt', 'Cost_Q': 'cost', 'Color_Q': 'color', 'Feedback_Slt': 'feedback', 'Feedback': 'feedback', 'Booking_Req': 'booking', 'Thanks': 'thanks', 'DualSim_Q': 'dualsim', 'Released_Yr_Q': 'year', 'OS_Q': 'os', 'Camera_Q': 'camera', 'Rating_Q': 'rating', 'Confirm_Q': 'confirm', 'Battery_Q': 'battery', 'Memory_Q': 'memory', 'DuslSim_Q': 'dualsim', 'Review_Q': 'review', 'Specification_Q': 'specification', 'Discount_Q': 'discount', 'Weight_g_Q': 'weight', 'Display_Size_Q': 'size', 'Radio_Q': 'radio', 'Internal_RAM_Q,RAM_Q': 'ram', 'P_Camera_Q': 'camera', 'Internal_RAM_Q': 'ram', 'Greet': 'greet', 'P_Camera_Q,S_Camera_Q': 'camera', 'Confirm': 'confirm', 'Released_Yr_Q,Released_Month_Q': 'month', 'RAM_Q,Internal_RAM_Q': 'ram', 'Phn_Info': 'phone', 'S_Camera_Q': 'camera', 'Sim_Q': 'sim', 'RAM_Q': 'ram', 'GPS_Q': 'gps', 'P_Camera,S_Camera_Q': 'camera', 'RAM,_Q,Internal_RAM_Q': 'ram', 'SIM_Q': 'sim'}
a_action_mp={'Specification_D': 'specification', 'Result': 'result', 'Cost_A': 'cost', 'Color_A ': 'color', 'Rephrase': 'rephrase', 'Success': 'success', 'Welcome': 'welcome', 'Color_A': 'color', 'DualSim_A': 'dualsim', 'Released_Yr_A': 'year', 'OS_A': 'os', 'Camera_A': 'camera', 'Rating_A': 'rating', 'Confirm': 'confirm', 'Battery_A': 'battery', 'Memory': 'memory', 'Review_A': 'review', 'Specification_A': 'specification', 'Discount_A': 'discount', 'Confirm_Q': 'confirm', 'Feedback': 'feedback', 'Feedback_R': 'feedback', 'Weight_g_A': 'weight', 'Display_Size_A': 'size', 'Radio_A': 'radio', 'Internal_RAM_A': 'ram', 'RAM_A': 'ram', 'P_Camera_A': 'camera', 'S_Camera_A': 'camera', 'ReleasedYear_A': 'year', 'ReleasedMonth_A': 'month', 'RAM_Q': 'ram', 'Greet': 'greet', 'P_Camera_Q': 'camera', 'Sim_A': 'sim', 'Cost_Q': 'cost', 'GPS_A': 'gps', 'Released_Month_A': 'month', 'Weight_g_Q': 'weight', 'P_Camera': 'camera', 'null': 'null', 'SIM_A': 'sim','Memory_A':'memory'} 
correct={'Camera': 'camera', 'Battery': 'battery', 'Color': 'color', 'OS': 'os', 'Dualsim': 'dual', 'Touch': 'touch', 'Memory': 'memory', 'Cost': 'cost', 'Rating': 'rating', 'Price': 'price', 'Released_Yr': 'year', 'Company': 'company', 'Model': 'model', 'Total_Review': 'review', 'DualSim': 'dual', 'RAM': 'ram', 'Display_Size': 'size', 'Weight_g': 'weight', 'Brand': 'brand', 'Radio': 'radio', 'Internal_RAM': 'ram', 'Sim': 'sim', 'P_Camera': 'camera', 'S_Camera': 'camera', 'Internal_Ram': 'ram', 'GPS': 'gps', 'Internal RAM': 'ram', 'Released_ Yr': 'year', 'Released_Month': 'month', 'SIM': 'sim', 'weight_g': 'weight', 'Weight_G': 'weight', 'Displa_Size': 'size', ' Internal_RAM': 'ram', 'nll': 'nll', 'battery': 'battery', 'Dsplay_Size': 'size', ' Model': 'model', 'Weight': 'weight', 'Apple': 'apple', ' P_Camera': 'camera'}

In [75]:
mem_arr=['4','6']
year_arr=['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
month_arr=['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']
rating_arr=['4 . 1','4 . 2','4 . 3','4 . 4','4 . 5','4 . 6','4 . 7','4 . 8','4 . 9','5 . 0','1','2','3','4','5'] 
bat_arr=['3800', '41000', '38000', '3000', '2400', '3300', '24', '3260', '1785', '2870', '2700', '7300', '32', '2900', '4000', '2620', '2850', '2300', '4010', '2950', '2500', '6020', '3505', '2730', '5000', '304', '2770', '4100', '1830', '2915', '2650', '1200', '2000', '4550', '2100', '1750', '400', '1960', '2050', '2800', '2600', '2530', '2750', '1950', '2110', '2910', '3020', '2760', '4800']
cam_arr=['13', '15', '10', '20', '16', '12', '23', '7', '5', '19', '8', '21', '4', '22', '64', '2', '1']
os_arr=['android','and','ios','mic','windows','blackberry','symphony','mm','coloros','fir']
color_arr=['black', 'mystic black', 'pure white', 'various', 'aquamarine green', 'graphite black', 'titan black', 'titanium black', 'silver', 'white', 'essential white', 'rock black', 'gentle black', 'titanium gray', 'blue', 'soft-touch black', 'satin gold', 'venus gold', 'halo gold', 'gold', 'volcanic grey', 'slate', 'yellow', 'midnight blue', 'lemon yellow', 'dark gray', 'full white', 'bluish black', 'mozilla orange', 'cloudy', 'red', 'green', 'cranberry pink', 'blue', 'flash red', 'twilight blue', 'reddish black', 'ardesia', 'dark blue', 'space gray', 'jet black', 'black(8', 'shimmer gold', 'navy black', 'glacier grey', 'glacier gray', 'gray', 'titanium grey', 'sand gold', 'sapphire black', 'glacier silver', 'charcoal black', 'rosa', 'pure black', 'rose gold', 'osmium black', 'meteorite black', 'burgundy red', 'sapphire black', 'crystal white', 'amethyst gray', 'royal blue', 'grey', 'neon orange', 'midnight black', 'mineral black', 'forest', 'champagne white', 'royal gold', 'champagne gold', 'brown', 'champagne gold', 'glossy black', 'cherry gold', 'jazz gold', 'mocha gold', 'rose gold', 'pearl white', 'quite black', 'amazing silver', 'brilliant black', 'stone black', 'black gold', 'gunmetal gray', 'carbon gray', 'dazzling gold', 'meteor gray', 'meteor grey', 'marble white', 'white birch', 'glacier blue', 'meridian gray', 'indigo black', 'santorini white', 'polar white', 'vanilla white', 'glacial silver', 'violet', 'radical yellow', 'stealth black', 'california blue', 'domino', 'active black', 'clearwater', 'active white', 'brown (teflon coating)', 'sophisticated black', 'yes', 'metal grey', 'amber gold', 'aurora blue', 'ceramic white', 'mystic silver', 'golden black', 'haze gold', 'graphite black', 'agate red', 'pink', 'prestige gold', 'obsidian black', 'moonlight silver', 'aluminium', 'luxurious gold', 'moonlight silver', 'obsidian black', 'snow white', 'crystal black', 'metallic black', 'crystal blue', 'matt black', 'shadow black', 'black body', 'orange', 'deep blue', 'icy white', 'dark blue', 'dark grey', 'metallic brown', 'aurora black', 'onyx black', 'gunmetal grey', 'matte black', 'platinum', 'matte white', 'tuxedo black', 'crimson', 'ebony', 'golden', 'titanium', 'flamingo red', 'mystic white', 'titan', 'indigo', 'titan silver', 'metallic black', 'space black', 'carbon', 'brilliant bronze', 'navy', 'blue', 'metallic gray', 'metallic silver', 'platinum silver', 'silk white', 'blue (boost mobile)', 'titan silver', 'purple', 'black & silver', 'champanage gold', 'mint green', 'black (front panel)', 'black front & white back', 'black (front)', 'white champagne', 'magnetic black', 'classy silver', 'cocoa brown', 'matte cyan', 'glossy cyan', 'iron gray', 'iron grey', 'licorice black', 'lunar gray', 'metallic cherry', 'lunar grey', 'black ballistic nylon', 'deep sea blue', 'black lens', 'champagne', 'licorice', 'cloud gray', 'electric magic black', 'golden bronze', 'black slate', 'midnight titanium', 'arte black', 'bright orange', 'glossy green', 'bright orange', 'bright green', 'cyan', 'lime green', 'matte black', 'steel dark', 'cool grey', 'nokia oro light', 'dark steel', 'frosty metal', 'silver grey', 'charcoal black', 'azure', 'carbon black', 'blue on white', 'chrome', 'white steel', 'black and cyan', 'gunmetal', 'graphite', 'onyx', 'sandstone black', 'obsidian', 'marine blue', 'steel blue', 'black sky', 'blue coral', 'camo green', 'black onyx', 'black sapphire', 'white frost', 'smoky titanium', 'white pearl', 'frosted white', 'titanium green', 'iris charcoal', 'electric blue', 'dazzling white', 'pebble blue', 'cream white', 'dive blue', 'black and gold', 'dark brown', 'luxury brown', 'urban grey', 'metallic blue', 'chic white', 'metallic gray and white', 'navy blue', 'carbon blue', 'noble black', 'metallic gray', 'black and grey', 'deep black', 'black and bronze', 'modern black', 'hanayome white', 'snowy white', 'luminous chrome', 'mineral black', 'serene black', 'classic black', 'blue gradient', 'iconic black', 'sensous black', 'elegant black', 'solid black', 'dark ale', 'sunburst', 'dark gray metallic', 'jet calf', 'onyx calf', 'jet', 'crown gold', 'ceramic black', 'coffee brown', 'moonstone grey', 'ion gold']
brand_arr=['xioami','apple','vodafone','iphone','redmi','zenfone','oppo','vivo','nokia','samsung','allview','htc','gionee','oneplus','accer','acer','sonim','qmobile','panasonic','verykool','huawei','asus','lenovo','coolpad','alcatelis','jio']
model_arr=['model aa', 'model b', 'model ff', 'model d', 'sure , model a', 'model h', 'model j', 'model ff', 'model j', 'model e', 'model k', 'model h', 'model h', 'model e', 'model b', 'model ff', 'model a', 'model ff', 'xplay6', 'iphone 7 plus', 'desire 10 pro', 'zuk z2 pro', 'model lumia 950', 'dtek60', 'moto maxx', 'liquid x2', 'galaxy j7 ( 2017 )', 'one max', 'a1', 'u ultra', 'pixel v2', 'mate 9 porsche design', 'liquid z520', 'one x10', 'galaxy c7 pro', 'x4 soul', 'galaxy c7 pro', 'redmi note prime', 'razr d1', 'zenfone 3 zoom ze553kl', 'moto maxx', 'f3 plus', 'galaxy c9 pro', 'r9s plus', 'f3', 'honor mx6', 'sure , v5s', 'xperia x performance', 'liquid jade primo', 'phone ms1', 'r11', 'k8 ( 2017 )', 'n3', 'galaxy c9 pro', 'v5s', 'xperia z4 tablet lte', '3t', 'galaxy j7 max', 'x9', 'v5s', 'a57', 'moto maxx', 'a57', 'galaxy c9 pro', 'x5max platinum edition', 'model xperia xa1 ultra', 'xperia xz premium', '2 3g', 'xperia xz premium', 'galaxy c7 pro', 'xperia xz', 'liquid jade primo', 'xperia l1', 'ms1', 'xperia xa1 ultra', 'mate 9', 'model desire 10 pro', 'xperia xz premium', 'mate 9', 'liquid x2', 'one e9+', 'one x10', 'marathon m5 plus', 'w909', 'zenfone live zb501kl', 'mate 9', 'axon lux', 'honor 7', 'galaxy a5 duos', 'a1', 'iconia talk s', 'model galaxy j3 emerge', 'liquid jade primo', 'note 5 lite', 'xperia xa ultra', 'xperia e4 dual', 'vibe p1m', 'xplay6', 'blade s7', 'x4 soul', 'zenfone 3s max zc521tl', 'pop 7 lte', 'dtek60', 'model nova 2', 'a59', 'r9 plus', 'model galaxy c9 pro', 'satio ( idou )', 'model mi note 2', 'galaxy j7 max', 'model galaxy c9 pro', 'galaxy s5 neo', 'model moto e3 power', 'droid turbo', 'model f1', 'model galaxy c7', 'r11 plus', 'signature touch ( 2015 )', 'then elife e8', 'mi 5c', 'blade a2 plus', 'galaxy j7 max', 'x7', 'a6600 plus', 'g6', 'm3 max', 'liquid x2', 'galaxy j3 ( 2017 )', 'one x10', 'hero cdma', 'moto x play dual sim', 'u11', '5', 'elife e8', 'zenfone 5 a500kl', 'iphone 7 plus', 'honor note 8', 'sure , g6', 'aurora', 's10', 's10', 'desire 830', 'galaxy s8', 'galaxy j7 ( 2016 )', 'v5 plus', 'redmi 3 pro', 'blade v8', 'axon 7 max', 'mi note 2', 'galaxy c9 pro', 'noir z8', 'xperia xa1 ultra', 'galaxy s7 active', "porsche design p'9982", '5', 'zenfone ar zs571kl', 'zenfone 3s max zc521tl', 'honor 9', 'nova 2', 'leap', 's10', 's9', 'pixel', 'moto e4 plus', 'eluga i3 mega', 'galaxy c7 pro', 'v3max', 'model y67', 'y67', 'idol 4s', 'blade a2 plus', 'a37', 'galaxy note7', 'galaxy a7 ( 2017 )', 'xperia c5 ultra dual', 'xperia xa dual', 'sure , model redmi note 4', 'mi 6', 'xplay5', 'z2', 'ms1', 'f1', 'f3 plus', 'liquid x2', 'zenfone 3 max zc553kl', 'lumia 630 dual sim', 'find 7a', 'galaxy s8+', 'galaxy s7', 'eluga ray x', 'xperia xz premium', 'mi note pro', 'iphone 6 plus', 'zenfone 3 laser zc551kl', 'z30', 'model butterfly 3', 'sure , f3', 'galaxy xcover 4', 'u ultra', 'one m9s', 'k6', 'g6', 'r7 plus', 'iconia talk s', 'x6s', 'blade qlux 4g', 'axon lux', 'noir e8', 'model zenfone live zb501kl', 'dtek50', 'nowadays s9', 'e7', 'model lumia 640 xl lte dual sim', 'model iphone 6s', 'sure , x5max platinum edition', 'model aurora', 'okay highway', 'ipaq hw6515', 'one m8s', 'desire 626g+', 'model honor 9', "model porsche design p'9983", 'r11', 'model moto e dual sim ( 2nd gen )', 'mc55', 'well , then v5s', 'model redmi 3 pro', 'p8 lite ( 2017 )', '6', 'galaxy xcover 4', 'sure , x9', 'moto e3 power', 'm2017', 'marathon m4', 'xperia z2a', 'zenfone 6 a601cg', 'aurora', 'iphone 7', 'galaxy a7 ( 2017 )', 'xperia e4g dual', 'xperia e5', 'xperia x', 'have a look at f5', 'sure , moto z2 play', 'iphone 7', 'conjr', 'fierce xl', 'p8 energy pro', 'v2 viper', 'blade v8', 'yutopia', '5 zs550kl', 'zenfone live zb501kl', 'a1 plus', 'cool s1', 'v10', 'r7 plus', 'gsmart classic lite', '1', 'k20 plus', 'mytouch q 2', 'canvas sliver 5 q450', 'f1 plus', 'xperia l1', '10 evo', 'honor 8 pro', 'e5', 'galaxy j2 prime', 'zenfone 2 deluxe ze551ml', 'marathon m5 plus', 'e6', 'eluga switch', 'm6s plus', 'sure , y7', 'sure , r3', 'moto z2 play', 'moto e4 plus ( usa )', 'canvas tab p666', 'p10 plus', 'le max 2', 'smart platinum 7', 'x7 plus', 'padfone infinity 2', 'a1 plus', 'x', 'eluga pulse', 'signature touch ( 2015 )', 'model v3', 'r9s', 'iphone 5s', 'pure xl', 'one ( m8 ) for windows', 'mate 9 pro', 'honor 3x pro', 'x10', 'model galaxy j1 ( 2016 )', 'galaxy c9 pro', 'honor magic', 'moto z force', 'light weight moto x ( 2nd gen )', 'one u11', 'iphone 7 plus', 'iphone 7', 'z30', 'zenfone go zb500kl', 'zenpad 3s 10 z500kl', '5 zs550kl', 'vivo iv', 'galaxy a7 ( 2017 )', 'lumia 2520', 'dtek50', 'galaxy c7', 'sure , w909', 'slate6 voicetab', 'model galaxy j7 prime', 'x venture', 'y28', 'blade s7', 'v5', 'tab prime 6', '3t', 'f3', 'one x10', 'x7', 'v3', 'xperia xa1 ultra', 'x6s', 'model xperia xa dual', 'xperia xa1', 'f3 plus', 'iphone 6s plus', 'model fire c', 'model zenfone ar zs571kl', 'priv', 'model iphone 7', 'galaxy s8', 'model s6 pro', 's9', 'model pioneer p6', 'model honor 8', 'model w909', 'blade v8', 'model blade g', 'iphone 6s', 'galaxy s8', 'model galaxy a5 ( 2017 )', 'x2 soul', 'mediapad m3 lite 10', '3', 's5015 spark ii', 'galaxy j3 emerge', 'galaxy note7', 'noir e8', 'galaxy xcover 4', 'mytouch q', 'canvas pulse 4g e451', 'pure xl', 's6', 'mi max 2', 'redmi 3 pro', 'redmi 3s', 'yutopia', 'pro 6 plus', 'iphone 5', '5 )', 'x3 soul pro', 'zenfone 5 a500kl', 'galaxy j7 max', 'galaxy j1 ace', 'galaxy j1 ace', 'galaxy s6 edge+', 'desire 728 dual sim', 'one m9s', 'a57', 'f1', 'eluga switch', 'pixel xl', 'one x10', 'z25', 'liquid z6 plus', 's9', 'honor 5a', 'model yoga tab 3 plus', 'u10', 'noir z8', 'xperia xa ultra', 'xp7', 'sl6010 cyprus lte', 'galaxy s8+']
ref_arr=['2012-','2012_','2020_','2020-','1212_','1212-'] 
quarter_to_month={'q4.released2009q4':'december','q1':'march','q2':'june','q3':'september','q4':'december','1q':'march','2q':'june','3q':'september','4q':'december'}

In [76]:
vocab_f=open('glove/glove.6B.50d.txt',encoding='utf-8')
word_to_id={} 
word_to_emb={}
val=4
word_to_id['<pad>']=3 
word_to_id['<unk>']=0
word_to_id['<start>']=1
word_to_id['<end>']=2
for line in vocab_f:
  word_to_id[line.split()[0]]=val 
  word_to_emb[line.split()[0]]=[float(i) for i in line.split()[1:]]  
  val+=1 
print(word_to_id['20']) 
print(len(word_to_emb['happy']))

328
50


In [77]:
word_to_emb['<unk>']=[0]*50 
word_to_emb['<pad>']=[0]*50 
temp_agent=[word_to_emb['<unk>']]  
import pandas as pd
df=pd.read_excel('data/SharedData.xls') 
User=[] 
User_Arr=[]
for row in df['User']:
    if type(row)==int:
        User.append([]) 
    else:
        User[-1].append(row.lower()) 
        User_Arr.append(row.lower()) 
Agent=[]
Agent_Arr=[]
for row in df['Agent']:
    if type(row)==int:
        Agent.append([]) 
    else:
        Agent[-1].append(row.lower()) 
        Agent_Arr.append(row.lower()) 
Intent=[]
Intent_Arr=[]
for row in df['Intent']:
    if type(row)==int:
        Intent.append([]) 
    else:
        Intent[-1].append(row.lower()) 
        Intent_Arr.append(row.lower()) 
Slot=[]
Slot_Arr=[]
Value=[]
Value_Arr=[]
temp_slot=[]
temp_val=[]
is_slot=[]
cnt=0
for row in df['Slot']:
    if type(row)==int:
        Slot.append([]) 
        Value.append([]) 
    elif type(row)==float:
        Slot[-1].append('null') 
        Value[-1].append('null')
        Slot_Arr.append(['null']) 
        Value_Arr.append(['null']) 
        temp_slot=['null'] 
        temp_val=['null']
    else: 
        temp_slot=[] 
        temp_val=[] 
        part=row.split(',') 
        for p in part:
          Slot[-1].append(correct[p.split('-')[0]]) 
          temp_slot.append(correct[p.split('-')[0]]) 
          if len(p.split('-'))>1:
            Value[-1].append(val_mp[p.split('-')[1].lower()]) 
            temp_val.append(val_mp[p.split('-')[1].lower()])
            if temp_val[-1]=='t':
                temp_val[-1]='t-mobile'
          else:
            Value[-1].append('null') 
            temp_val.append('null') 
    if type(row)!=int and type(row)!=float:
      Slot_Arr.append([]) 
      for a in temp_slot:
        Slot_Arr[-1].append(a)  
      Value_Arr.append([]) 
      for a in temp_val:
        Value_Arr[-1].append(a)    
A_Action=[]
A_Action_Arr=[]
for row in df['A_Action']:
    if type(row)==int:
        A_Action.append([])
    elif type(row)==float:
        A_Action[-1].append('null') 
    else:
        row=row.split(',')[0]
        A_Action[-1].append(a_action_mp[row]) 
        A_Action_Arr.append(a_action_mp[row]) 
U_Action=[]
U_Action_Arr=[]
for row in df['U_Action']:
    if type(row)==int:
        U_Action.append([])
    elif type(row)==float:
        U_Action[-1].append('null') 
        U_Action_Arr.append('null')  
    else:
        U_Action[-1].append(u_action_mp[row]) 
        U_Action_Arr.append(u_action_mp[row]) 

In [78]:
import pandas as pd
xl=pd.ExcelFile('data/database.xls')
df=pd.DataFrame() 
df=xl.parse(xl.sheet_names[1])
db=[]
for i in range(0,len(df['Serial'])):
  j=i
  db.append({'ref_no':'','cost_price':'','money_unit':'','month_name':'','year_name':'','review_person':'3500','rating_score':'4.4','cam_val':'','mem_size':'','bat_size':'','color_name':'','os_name':'','model_name':'','brand_name':''})
  db[j]['ref_no']=str(j)
  for k in df.keys():
    if k=='brand':
      db[j]['brand_name']=df[k][j].lower()
    if k=='model':
      db[j]['model_name']=str(df[k][j]).lower() 
    if k=='Battery':
      if df[k][j]<100:
        df[k][j]*=100 
      if df[k][j]>50000:
        df[k][j]=df[k][j]%1000
      db[j]['bat_size']=df[k][j]
    if k=='P_Camera':
      db[j]['cam_val']=df[k][j]
      if type(df[k][j])==str and 'mp' in df[k][j].lower():
            db[j]['cam_val']=df[k][j][:-2]
    if k=='OS':
      db[j]['os_name']=df[k][j].lower() 
    if k=='Color':
      db[j]['color_name']=df[k][j].lower() 
    if k=='Internal_RAM':
      db[j]['mem_size']=df[k][j] 
    if k=='approx_price_EUR':
      db[j]['cost_price']=df[k][j] 
      db[j]['money_unit']='euro' 
    if k=='ReleasedYear':
      db[j]['year_name']=df[k][j] 
    if k=='ReleasedMonth':
      db[j]['month_name']=df[k][j].lower()
      month_flag=0
      for month in month_arr:
        if month in db[j]['month_name']:
            db[j]['month_name']=month
            month_flag=1 
            break 
      if month_flag==0:
        db[j]['month_name']=quarter_to_month[db[j]['month_name']]
    if k=='weight_g':
      db[j]['weight_val']=df[k][j] 
    if k=='Radio':
      if df[k][j]=='Y':
        tmp='yes' 
      else:
        tmp='no'
      db[j]['rad_bin']=tmp 
    if k=='Display_Size':
      db[j]['size_val']=df[k][j]  
len(db)

c:\users\saswat\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\saswat\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2517

In [79]:
temp_col_mp={} 
common_color=['black','white','red','blue','gold','silver','pink','orange','titanium','cyan','chrome','charcoal','bronze','gunmetal','gray','grey','yellow','brown','dark','calf','light','green','metallic','frosty','clearwater','platinum','aluminium']
for data in db:
    if data['color_name'] not in temp_col_mp:
        ans=data['color_name'] 
        for col in common_color:
            if col in ans:
                ans=col 
                break
        if ans=='ardesia' or ans=='allview':
            ans='slate'
        temp_col_mp[data['color_name']]=ans 
word_to_emb['allview']=[((float(word_to_emb['acer'][i])+float(word_to_emb['asus'][i]))/2) for i in range(50)]
word_to_emb['celkon']=[((float(word_to_emb['zte'][i])+float(word_to_emb['meizu'][i]))/2) for i in range(50)]
word_to_emb['coolpad']=[((float(word_to_emb['vivo'][i])+float(word_to_emb['oppo'][i]))/2) for i in range(50)]
word_to_emb['garmin-asus']=word_to_emb['asus']
word_to_emb['gionee']=[((float(word_to_emb['xiaomi'][i])+float(word_to_emb['vivo'][i]))/2) for i in range(50)] 
word_to_emb['yezz']=[((float(word_to_emb['zte'][i])+float(word_to_emb['pantech'][i]))/2) for i in range(50)]
word_to_emb['i-mate']=[((float(word_to_emb['kyocera'][i])+float(word_to_emb['jolla'][i]))/2) for i in range(50)]
word_to_emb['intex']=[((float(word_to_emb['micromax'][i])+float(word_to_emb['lava'][i]))/2) for i in range(50)]
word_to_emb['leeco']=[((float(word_to_emb['intex'][i])+float(word_to_emb['i-mate'][i]))/2) for i in range(50)]
word_to_emb['oneplus']=[((float(word_to_emb['apple'][i])+float(word_to_emb['samsung'][i]))/2) for i in range(50)]
word_to_emb['karbonn']=word_to_emb['karbon']
word_to_emb['prestigio']=[((float(word_to_emb['apple'][i])+float(word_to_emb['asus'][i]))/2) for i in range(50)]
word_to_emb['qmobile']=[((float(word_to_emb['nokia'][i])+float(word_to_emb['oppo'][i]))/2) for i in range(50)]
word_to_emb['verykool']=[((float(word_to_emb['coolpad'][i])+float(word_to_emb['lg'][i]))/2) for i in range(50)]
word_to_emb['wiko']=[((float(word_to_emb['oppo'][i])+float(word_to_emb['asus'][i]))/2) for i in range(50)]
word_to_emb['sonim']=[((float(word_to_emb['sony'][i])+float(word_to_emb['samsung'][i]))/2) for i in range(50)]
word_to_emb['sony ericsson']=word_to_emb['ericsson'] 

tmp_mp={}
cnt=0
for i in range(len(Slot_Arr)):
    for j in range(len(Slot_Arr[i])):
        if Slot_Arr[i][j]=='brand' and Value_Arr[i][j] not in tmp_mp:
            tmp_mp[Value_Arr[i][j]]=cnt
            cnt+=1


In [80]:
db_out=[] 
db_in=[]
cnt_temp=0
key_to_pos={}
for data in db:
    db_out.append([]) 
    for i in range(2517):
        if i==cnt_temp:
            db_out[-1].append(1) 
        else:
            db_out[-1].append(0) 
    cnt_temp+=1
    db_in.append([])
    db_in[-1].append(word_to_emb[str(data['brand_name'])]) 
    key_to_pos['brand']=0
    if data['cost_price']>1000:
        data['cost_price']=(data['cost_price']//100)*100
    if data['cost_price']>10000:
        data['cost_price']=(data['cost_price']//100)
    db_in[-1].append(word_to_emb[str(data['cost_price'])])
    key_to_pos['cost']=1
    db_in[-1].append(word_to_emb[str(data['month_name'])])
    key_to_pos['month']=2
    db_in[-1].append(word_to_emb[str(data['year_name'])])
    key_to_pos['year']=3
    db_in[-1].append(word_to_emb[str(data['cam_val']).lower()]) 
    key_to_pos['camera']=4
    if type(data['mem_size'])==str and 'GB' in data['mem_size']:
        data['mem_size']=data['mem_size'][:-2]
    db_in[-1].append(word_to_emb[str(int(data['mem_size'])//1)])
    key_to_pos['memory']=5
    temp_sz=isNan(data['bat_size'])
    key_to_pos['battery']=6
    if temp_sz>10000:
        temp_sz=(temp_sz//1000)*1000
    db_in[-1].append(word_to_emb[str((temp_sz//100)*100)])
    db_in[-1].append(word_to_emb[str(temp_col_mp[data['color_name']])])
    key_to_pos['color']=7
    db_in[-1].append(word_to_emb[str(data['os_name'])])
    key_to_pos['os']=8
    db_in[-1].append(word_to_emb[str(data['rad_bin'])])
    key_to_pos['radio']=9
    db_in[-1].append(word_to_emb[str(data['size_val'])])
    key_to_pos['display']=10
    if type(data['weight_val'])==str:
        data['weight_val']=data['weight_val'].split('-')[0]
    key_to_pos['weight']=11
    db_in[-1].append(word_to_emb[str(int(data['weight_val'])//1)])
print(len(db_in[0]))

12


In [81]:
inp_span=[] 
out_span=[]
cnt=0
for i in range(2517):
    cnt+=1 
    inp_span.append([])
    for j in range(12):
      inp_span[-1].append(db_in[i][j])
for p in range(10):
    for i in range(2517):
      cnt+=1 
      inp_span.append([])
      for j in range(12):
        if j==12-p:
          inp_span[-1].append([0]*50) 
        else:
          inp_span[-1].append(db_in[i][j]) 
    for q in range(p+1,11):
        for i in range(2517):
          cnt+=1
          inp_span.append([]) 
          for j in range(12):
            if j==p or j==q:
              inp_span[-1].append(db_in[i][j]) 
            else:
              inp_span[-1].append([0]*50) 
        for i in range(2517):
          cnt+=1
          inp_span.append([]) 
          for j in range(12):
            if j==11-p or j==11-q:
              inp_span[-1].append([0]*50) 
            else:
              inp_span[-1].append(db_in[i][j]) 
        for r in range(q+1,12):
            for i in range(2517):
                cnt+=1
                inp_span.append([])
                for j in range(12):
                    if j==p or j==q or j==r:
                        inp_span[-1].append(db_in[i][j]) 
                    else:
                        inp_span[-1].append([0]*50) 
                
print(cnt)
for i in range(150000):
  out_span.append(db_out[i%2517]) 

858297


In [82]:
nltk.download('punkt')
def padd(string):
  arr=word_tokenize(string)  
  l=len(arr) 
  for i in range(max(0,12-l)):
    arr.append('<pad>') 
  return arr[:12]  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Saswat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [83]:
temp=[]
for m in range(2000,2020):
  temp.append(str(m)) 
print(temp)

['2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']


In [84]:
for i in range(4201):
  a=word_tokenize(Agent_Arr[i])
  for j in range(len(a)):
    st=a[j]
    #print(st)
    for k in range(len(a[j])):
      if a[j][k]=='.' and len(a[j])!=1 and a[j][k-1]!=' ':
        st=a[j][:k]+' '+'.'+' '+a[j][k+1:] 
        break 
    if 'mah' in st and len(st)>3:
      st=st[:-3]+' '+st[-3:] 
    for r in ref_arr:
      if r in st: 
        st='ref_no'
        break
    if st in clean_mp:
      st=clean_mp[st]  
    a[j]=st 
  Agent_Arr[i]=' '.join(a)
print(Agent_Arr[i])

you are welcome . i am happy to help . do visit us again .


In [85]:
db_dummy=[] 
org_Agent_Arr=[] 
for i in range(4201):
  org_Agent_Arr.append(Agent_Arr[i]) 
for i in range(4201):
  if U_Action_Arr[i]=='rec_slt' or U_Action_Arr[i]=='feedback' or U_Action_Arr[i]=='confirm':
    for model in model_arr:
      if model in Agent_Arr[i]:
          Agent_Arr[i]=Agent_Arr[i].replace(model,'model_name') 
    for brand in brand_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(brand,'brand_name') 
  if U_Action_Arr[i]=='battery':
    for bat in bat_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(bat,'bat_size') 
  if U_Action_Arr[i]=='camera':
    for cam in cam_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(cam,'cam_val') 
  if U_Action_Arr[i]=='color':
    for color in color_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(color,'color_name') 
  if U_Action_Arr[i]=='os':
    for os in os_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(os,'os_name') 
  if U_Action_Arr[i]=='memory':
    for mem in mem_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(mem,'mem_size') 
  if U_Action_Arr[i]=='specification' or U_Action_Arr[i]=='feedback':
    a=word_tokenize(Agent_Arr[i])
    for j in range(len(a)):
      st=a[j] 
      if st=='mp':
        a[j-1]='cam_val' 
      elif st=='gb':
        a[j-1]='mem_size' 
      elif st=='mah':
        a[j-1]='bat_size' 
    Agent_Arr[i]=' '.join(a) 
  if U_Action_Arr[i]=='review' or U_Action_Arr[i]=='rating':
    for r in rating_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(r,'rating_score') 
    a=word_tokenize(Agent_Arr[i]) 
    for j in range(len(a)):
      if isint(a[j])==1 and int(a[j])>1000:
        a[j]='review_person' 
    Agent_Arr[i]=' '.join(a) 
  if U_Action_Arr[i]=='month' or U_Action_Arr[i]=='year':
    for m in month_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(m,'month_name') 
    for y in year_arr:
      Agent_Arr[i]=Agent_Arr[i].replace(y,'year_name') 
  if U_Action_Arr[i]=='weight':
    a=word_tokenize(Agent_Arr[i]) 
    for j in range(len(a)):
      if isint(a[j])==1:
        a[j]='weight_val'
        break 
    Agent_Arr[i]=' '.join(a) 
  if U_Action_Arr[i]=='size':
    a=word_tokenize(Agent_Arr[i]) 
    for j in range(len(a)):
      if isint(a[j])==1:
        a[j]='size_val' 
        break
    Agent_Arr[i]=' '.join(a) 
  if U_Action_Arr[i]=='cost':
    a=word_tokenize(Agent_Arr[i]) 
    for j in range(len(a)):
      if isint(a[j])==1 and int(a[j])>100:
        a[j]='cost_price' 
        break
    Agent_Arr[i]=' '.join(a)
    for u in ['euro','rs','rupees']:
      Agent_Arr[i]=Agent_Arr[i].replace(u,'money_unit') 
    
print(Agent_Arr[1]) 
print(org_Agent_Arr[1])

model_name by brand_name is one of the best phone for camera as well as battery
model aa by oppo is one of the best phone for camera as well as battery


In [86]:
name_to_slot={'ref_no':'reference','cost_price':'price','money_unit':'money','month_name':'month','year_name':'year','review_person':'review','rating_score':'rating','cam_val':'camera','mem_size':'memory','bat_size':'battery','color_name':'color','os_name':'os','model_name':'model','brand_name':'brand','weight_val':'weight','size_val':'size'} 
slot_to_key={} 
for k,v in name_to_slot.items():
  if v not in slot_to_key:
    slot_to_key[v]=k 


In [87]:
import nltk
nltk.download('punkt')
for i in range(4201):
  a=word_tokenize(User_Arr[i])
  for j in range(len(a)):
    st=a[j]
   
    for k in range(len(a[j])):
      if a[j][k]=='.':
        st=a[j][:k]+' '+'.'+' '+a[j][k+1:] 
        break 
    a[j]=st 
  User_Arr[i]=' '.join(a)
print(User_Arr[i])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Saswat\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


thanks dear


In [88]:
user_arr=[]
for dial in User_Arr:
  user_arr.append(padd(dial))  
user_emb=[] 
sm=[0]*50 
for dial in user_arr:
  user_emb.append([])
  for word in dial:
    if word in word_to_emb:
      user_emb[-1].append(word_to_emb[word])
    else:
      user_emb[-1].append(word_to_emb['<unk>']) 
print(len(user_emb)) 

4201


In [89]:
agent_arr=[]
for dial in Agent_Arr:
  agent_arr.append(padd(dial))  
print(agent_arr)
agent_emb=[] 
for dial in agent_arr:
  agent_emb.append([]) 
  for word in dial:
    if word in word_to_emb:
      agent_emb[-1].append(word_to_emb[word])
    else:
      agent_emb[-1].append(word_to_emb['<unk>']) 
print(agent_emb[1])
print(len(agent_emb)) 

[['hey', ',', 'sure', ',', 'what', 'kind', 'of', 'phone', 'do', 'you', 'need', '?'], ['model_name', 'by', 'brand_name', 'is', 'one', 'of', 'the', 'best', 'phone', 'for', 'camera', 'as'], ['its', 'cost', 'is', 'money_unit', 'cost_price', 'only', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'], ['the', 'color', 'of', 'oppo', 'aa', 'is', 'color_name', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'], ['sir', 'this', 'model_nameas', 'strong', 'features', 'but', 'available', 'in', 'white', 'only', '.', '<pad>'], ['you', 'can', 'take', 'model_name', 'by', 'brand_name', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'], ['it', "'s", 'of', 'cost_price', '.', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'], ['yes', 'done', '.', 'i', 'am', 'booking', 'it', 'for', 'you', '.', '<pad>', '<pad>'], ['welcome', 'and', 'thank', 'you', 'so', 'much', 'for', 'shopping', 'with', 'us', '.', '<pad>'], ['hello', ',', 'what', 'kind', 'of', 'phone', 'do', 'you', 'want', '?', '<pad>', '<pad>

In [90]:
slot=[] 
mp_sl={}
cnt=0
for row in Slot:
  for a in row:
    if a not in mp_sl:
      mp_sl[a]=cnt 
      cnt+=1 
      slot.append(a) 
print(cnt)
slot_arr=[] 
for row in Slot_Arr:
  slot_arr.append([]) 
  for i in range(cnt):
    slot_arr[-1].append(0) 
  for slt in row:
    slot_arr[-1][mp_sl[slt]]=1  
print(len(slot_arr))
print(slot_arr[996])

25
4201
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [91]:
value=[] 
mp_vl={}
cnt=0
for row in Value:
  for a in row:
    if a not in mp_vl:
      mp_vl[a]=cnt 
      cnt+=1 
      value.append(a)
mp_vl['t-mobile']=cnt 
cnt+=1 
print(cnt) 
value_arr=[] 
for row in Value_Arr:
  value_arr.append([]) 
  for i in range(cnt):
    value_arr[-1].append(0) 
  for slt in row:
    value_arr[-1][mp_vl[slt]]=1  
print(len(value_arr))

153
4201


In [92]:
ag_action=[] 
mp_ag={}
cnt=0
for row in A_Action:
  for a in row:
    if a not in mp_ag:
      mp_ag[a]=cnt 
      cnt+=1 
      ag_action.append(a) 
print(cnt)
mp_ag_ag={} 
for k in mp_ag:
  mp_ag_ag[k]=[] 
  for i in range(26):
    if mp_ag[k]==i:
      mp_ag_ag[k].append(1) 
    else:
      mp_ag_ag[k].append(0) 
ag_action_arr=[] 
for row in A_Action:
  for a in row:
    ag_action_arr.append(mp_ag_ag[a]) 
print(len(ag_action_arr))

26
4201


In [93]:
ug_action=[] 
mp_ug={}
cnt=0
for row in U_Action:
  for a in row:
    if a not in mp_ug:
      mp_ug[a]=cnt 
      cnt+=1 
      ug_action.append(a) 
print(cnt)
mp_ug_ug={} 
for k in mp_ug:
  mp_ug_ug[k]=[] 
  for i in range(cnt):
    if mp_ug[k]==i:
      mp_ug_ug[k].append(1) 
    else:
      mp_ug_ug[k].append(0) 
ug_action_arr=[] 
for row in U_Action:
  for a in row:
    ug_action_arr.append(mp_ug_ug[a]) 
print(len(ug_action_arr))

27
4201


In [94]:
for i in db[0]:
  word_to_emb[i]=word_to_emb['value']

In [95]:
first_arr=[] 
first_emb=[] 
first_mp=[]
first=[]
cnt=[]
for k in range(12):
  first_arr.append([]) 
  first_emb.append([]) 
  first_mp.append({}) 
  first.append([])
  cnt.append(0) 
  for res in agent_arr:
    if res[k] not in first_mp[k]:
      first_mp[k][res[k]]=cnt[k] 
      first[k].append(res[k])
      cnt[k]+=1
  print(cnt[k]) 
  for i in range(4201):
    first_arr[k].append([])
    if agent_arr[i][k] in word_to_emb:
      first_emb[k].append(word_to_emb[agent_arr[i][k]]) 
    else:
      first_emb[k].append(word_to_emb['place'])
    for j in range(cnt[k]):
      if first_mp[k][agent_arr[i][k]]==j:
        first_arr[k][i].append(1)
      else:
        first_arr[k][i].append(0) 
  print(first_arr[k][0])

190
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
227
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
251
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [96]:
org_agent_arr=[] 
for i in range(4201):
  org_agent_arr.append(padd(org_Agent_Arr[i]))
inp=[]
for k in range(12): 
  inp.append([])
  for i in range(4201):
    inp[k].append([]) 
    for j in range(12):
      if user_arr[i][j] in word_to_emb:
        inp[k][i].append(word_to_emb[user_arr[i][j]]) 
      else:
        inp[k][i].append(word_to_emb['<unk>'])
print(len(inp[0][0]))
for k in range(1,12):
  for i in range(4201):
    for j in range(k):
      if agent_arr[i][j] in word_to_emb:
        inp[k][i].append(word_to_emb[agent_arr[i][j]])
      elif agent_arr[i][j] in clean_mp:
        inp[k][i].append(word_to_emb[clean_mp[agent_arr[i][j]]])
      else:
        inp[k][i].append(word_to_emb['place'])

12


In [97]:
value_on_slot={} 
values_of_slot={}
for i in range(len(slot)):
  value_on_slot[slot[i]]={} 
  values_of_slot[slot[i]]=[]
for i in range(4201):
  for j in range(len(Slot_Arr[i])):
    if Value_Arr[i][j] not in values_of_slot[Slot_Arr[i][j]]:
      values_of_slot[Slot_Arr[i][j]].append(Value_Arr[i][j]) 
for k,v in values_of_slot.items():
  for value in v:
    value_on_slot[k][value]=[0]*len(v) 
    value_on_slot[k][value][v.index(value)]=1 


In [98]:
inp_slot=[] 
out_slot=[]
for i in range(4201):
  for j in range(len(Slot_Arr[i])):
    out_slot.append([])
    inp_slot.append([]) 
    for k in range(12):
      inp_slot[-1].append(user_emb[i][k]) 
    for k in range(25):
      if mp_sl[Slot_Arr[i][j]]==k:
        out_slot[-1].append(1) 
      else:
        out_slot[-1].append(0) 

In [99]:
inp_val=[] 
out_val=[]
for i in range(4201):
  for j in range(len(Value_Arr[i])):
    out_val.append([])
    inp_val.append([]) 
    for k in range(12):
      inp_val[-1].append(user_emb[i][k]) 
    inp_val[-1].append(word_to_emb[slot[np.argmax(out_slot[len(inp_val)-1])]]) 
    for k in range(len(value)):
      if mp_vl[Value_Arr[i][j]]==k:
        out_val[-1].append(1) 
      else:
        out_val[-1].append(0) 

In [100]:
copy_arr=[]
copy_slot={}
copy_value={}
for i in range(25):
  copy_slot[slot[i]]=[] 
  copy_value[slot[i]]=[] 
for i in range(4201):
  words=word_tokenize(User_Arr[i])
  for j in range(len(Slot_Arr[i])):
    for word in words:
      if word in word_to_emb:
        copy_slot[Slot_Arr[i][j]].append(word_to_emb[word]) 
      else:
        copy_slot[Slot_Arr[i][j]].append(word_to_emb['blank'])
      if Value_Arr[i][j]==word:
        copy_value[Slot_Arr[i][j]].append([0,1]) 
      else:
        copy_value[Slot_Arr[i][j]].append([1,0])  

In [101]:
for k in value_on_slot:
  for v in value_on_slot[k]:
    if len(value_on_slot[k][v])==1:
      value_on_slot[k][v]=[1,0]
value_slot={}
value_value={}
for i in range(25):
  value_slot[slot[i]]=[] 
  value_value[slot[i]]=[] 
for i in range(4201):
  words=word_tokenize(User_Arr[i])
  for j in range(len(Slot_Arr[i])):
    value_slot[Slot_Arr[i][j]].append(user_emb[i]) 
    value_value[Slot_Arr[i][j]].append(value_on_slot[Slot_Arr[i][j]][Value_Arr[i][j]]) 
value_value['null'][0]

[1, 0]

In [102]:
def isstr(word):
  try:
    word=int(word) 
    word=float(word) 
  except:
    word=word 
  if type(word)==str:
    return 1 
  else:
    return 0
temp={}
for i in range(4201):
  if U_Action_Arr[i] not in temp:
    temp[U_Action_Arr[i]]=i 
print(temp) 
temp=[]
for i in range(4201):
  for word in agent_arr[i]:
    if word not in word_to_id and word not in brand_arr and word!='ref_no':
      if isstr(word)==1:
        temp.append(word) 
tmp_mp={} 
for t in temp:
  if 'gms' in t:
    tmp_mp[t]=1 
print(tmp_mp)
print(temp)

{'phone': 0, 'rec_slt': 1, 'cost': 2, 'color': 3, 'feedback': 4, 'booking': 7, 'thanks': 8, 'dualsim': 21, 'year': 22, 'os': 23, 'camera': 31, 'rating': 32, 'confirm': 33, 'battery': 41, 'memory': 53, 'review': 74, 'specification': 95, 'discount': 99, 'weight': 244, 'size': 253, 'radio': 269, 'ram': 276, 'greet': 302, 'month': 372, 'sim': 730, 'gps': 950, 'null': 2335}
{}
['model_name', 'brand_name', 'money_unit', 'cost_price', 'color_name', 'model_nameas', 'model_name', 'brand_name', 'cost_price', 'model_name', 'brand_name', 'money_unit', 'cost_price', 'color_name', 'model_namey', 'brand_name', 'money_unit', 'cost_price', 'model_name', 'money_unit', 'cost_price', 'year_name', 'os_name', 'model_name', 'brand_name', 'money_unit', 'cost_price', 'model_name', 'brand_name', 'cam_val', 'rating_score', 'cost_price', 'model_name', 'brand_name', 'model_name', 'brand_name', 'bat_size', 'color_name', 'year_name', 'money_unit', 'cost_price', 'model_name0', 'model_name', 'brand_name', 'money_unit'

In [103]:
def arg3max(arr):
  temp_arr=[] 
  for i in range(len(arr)):
    temp_arr.append((arr[i],i)) 
  temp_arr.sort() 
  temp_arr.reverse() 
  ans_arr=[] 
  for i in range(3):
    ans_arr.append((temp_arr[i][0],temp_arr[i][1],0)) 
  return ans_arr 
def combineArg3max(arr,p):
  temp_arr=[] 
  for i in range(len(arr[0])):
    temp_arr.append((100*p[0]*arr[0][i],i,0))  
    temp_arr.append((100*p[1]*arr[1][i],i,1))
    temp_arr.append((100*p[2]*arr[2][i],i,2))
  temp_arr.sort() 
  temp_arr.reverse() 
  ans_arr=[] 
  for i in range(3):
    ans_arr.append(temp_arr[i])   
  return ans_arr
def beam_search(inp_arr,model,first):
  inp_arr_temp=[] 
  for i in range(12):
    inp_arr_temp.append(inp_arr[i]) 
  dial_arr=[[],[],[]] 
  p=[]
  out0=model[0].predict(np.array([inp_arr])) 
  temp=arg3max(out0[0])
  temp_dial=[]
  for i in range(3): 
    dial_arr[i].append(first[0][temp[i][1]])  
  for i in range(3):
    p.append(temp[i][0]) 
  for j in range(1,12):
    temp_arr=[]
    for i in range(3):
      for k in range(len(dial_arr[i])):
       
        if dial_arr[i][k] in word_to_emb:
          inp_arr_temp.append(word_to_emb[dial_arr[i][k]]) 
        else:
          inp_arr_temp.append(word_to_emb['place'])
      temp_arr.append(model[j].predict(np.array([inp_arr_temp]))[0]) 
      for k in range(len(dial_arr[i])):
        inp_arr_temp.pop() 
    temp=combineArg3max(temp_arr,p) 
    temp_dial=[]
    for i in range(3): 
      dial_arr[temp[i][2]].append(first[j][temp[i][1]])
      temp_dial.append([]) 
      for k in range(len(dial_arr[temp[i][2]])):
        temp_dial[i].append(dial_arr[temp[i][2]][k]) 
      dial_arr[temp[i][2]].pop()   
    for i in range(3):
      dial_arr[i]=temp_dial[i]
    for i in range(3):
      p[i]=temp[i][0]  
  return ' '.join(dial_arr[0]) 
def whole_beam_search(inp_arr,model,first):
  output=[]
  for i in range(len(inp_arr)):
    output.append(beam_search(inp_arr[i],model,first)) 
  return output 


In [104]:
print(User_Arr[0])
 

hi , can you help me in buying a phone ?


In [105]:
def model1(X,Y,input_size1,input_size2,output_size):
  m=Sequential()
  m.add(Bidirectional(LSTM(256,input_shape=(None,input_size2))))
  m.add(Dropout(0.5))
  m.add(Dense(512,activation='relu'))
  m.add(Dense(1024,activation='relu'))
  m.add(Dropout(0.5))
  m.add(Dense(output_size,activation='softmax'))
  m.compile('Adam','categorical_crossentropy',['accuracy'])
  m.fit(X,Y,epochs=3)
  return m 

In [106]:
def model2(X,Y,input_size,output_size):
  m=Sequential() 
  m.add(Dense(32,activation='relu'))
  m.add(Dense(output_size,activation='softmax')) 
  m.compile('Adam','categorical_crossentropy',['accuracy'])
  m.fit(X,Y,epochs=10)
  return m 

In [107]:
copy_model={} 
for i in range(25):
  copy_model[slot[i]]=model2(np.array(copy_slot[slot[i]]),np.array(copy_value[slot[i]]),50,2)


Epoch 1/10
693/693 [==============================] - 4s 1ms/step - loss: 0.0584 - accuracy: 0.9948
Epoch 2/10
693/693 [==============================] - 1s 1ms/step - loss: 3.2332e-04 - accuracy: 1.0000: 0s - loss: 3.4641e-04 - accuracy: 
Epoch 3/10
693/693 [==============================] - 1s 950us/step - loss: 9.6689e-05 - accuracy: 1.0000
Epoch 4/10
693/693 [==============================] - 1s 912us/step - loss: 4.1443e-05 - accuracy: 1.0000
Epoch 5/10
693/693 [==============================] - 1s 835us/step - loss: 2.0418e-05 - accuracy: 1.0000
Epoch 6/10
693/693 [==============================] - 1s 835us/step - loss: 1.2633e-05 - accuracy: 1.0000
Epoch 7/10
693/693 [==============================] - 1s 858us/step - loss: 4.7125e-06 - accuracy: 1.0000
Epoch 8/10
693/693 [==============================] - 1s 883us/step - loss: 3.0160e-06 - accuracy: 1.0000
Epoch 9/10
693/693 [==============================] - 1s 981us/step - loss: 2.8910e-06 - accuracy: 1.0000
Epoch 10/10
693/69

5/5 [==============================] - 0s 1ms/step - loss: 0.1093 - accuracy: 0.9691
Epoch 6/10
5/5 [==============================] - 0s 2ms/step - loss: 0.1101 - accuracy: 0.9639
Epoch 7/10
5/5 [==============================] - 0s 2ms/step - loss: 0.1015 - accuracy: 0.9652
Epoch 8/10
5/5 [==============================] - 0s 1ms/step - loss: 0.0975 - accuracy: 0.9630
Epoch 9/10
5/5 [==============================] - 0s 2ms/step - loss: 0.0920 - accuracy: 0.9622
Epoch 10/10
5/5 [==============================] - 0s 1ms/step - loss: 0.1135 - accuracy: 0.9444
Epoch 1/10
76/76 [==============================] - 0s 1ms/step - loss: 0.6981 - accuracy: 0.5565
Epoch 2/10
76/76 [==============================] - 0s 929us/step - loss: 0.1198 - accuracy: 0.9609
Epoch 3/10
76/76 [==============================] - 0s 943us/step - loss: 0.0701 - accuracy: 0.9797
Epoch 4/10
76/76 [==============================] - 0s 954us/step - loss: 0.0466 - accuracy: 0.9827
Epoch 5/10
76/76 [==================

98/98 [==============================] - 0s 967us/step - loss: 0.0259 - accuracy: 0.9892
Epoch 10/10
98/98 [==============================] - 0s 944us/step - loss: 0.0182 - accuracy: 0.9933
Epoch 1/10
45/45 [==============================] - 0s 1ms/step - loss: 0.6368 - accuracy: 0.6114
Epoch 2/10
45/45 [==============================] - 0s 958us/step - loss: 0.1824 - accuracy: 0.9177
Epoch 3/10
45/45 [==============================] - 0s 940us/step - loss: 0.1104 - accuracy: 0.9460
Epoch 4/10
45/45 [==============================] - 0s 1ms/step - loss: 0.0847 - accuracy: 0.9780
Epoch 5/10
45/45 [==============================] - 0s 1ms/step - loss: 0.0696 - accuracy: 0.9778
Epoch 6/10
45/45 [==============================] - 0s 1ms/step - loss: 0.0660 - accuracy: 0.9764
Epoch 7/10
45/45 [==============================] - 0s 1ms/step - loss: 0.0593 - accuracy: 0.9755
Epoch 8/10
45/45 [==============================] - 0s 956us/step - loss: 0.0633 - accuracy: 0.9733
Epoch 9/10
45/45 [==

1/1 [==============================] - 0s 2ms/step - loss: 0.2903 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 0s 4ms/step - loss: 0.2666 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 0s 4ms/step - loss: 0.2449 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 0s 4ms/step - loss: 0.2252 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 0s 4ms/step - loss: 0.2073 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 0s 3ms/step - loss: 0.1910 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 0s 2ms/step - loss: 0.1761 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 0s 4ms/step - loss: 0.1625 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 0s 3ms/step - loss: 0.1501 - accuracy: 1.0000


In [108]:
out=copy_model['battery'].predict(np.array(user_emb[1]))
print(out,User_Arr[1])

[[9.9902248e-01 9.7748532e-04]
 [9.9944681e-01 5.5316446e-04]
 [9.9987733e-01 1.2270815e-04]
 [1.2190204e-01 8.7809789e-01]
 [9.9997628e-01 2.3672435e-05]
 [9.9904197e-01 9.5799286e-04]
 [9.9960321e-01 3.9680683e-04]
 [9.9772328e-01 2.2766751e-03]
 [7.1152478e-01 2.8847528e-01]
 [7.1152478e-01 2.8847528e-01]
 [7.1152478e-01 2.8847528e-01]
 [7.1152478e-01 2.8847528e-01]] it should have good camera and battery capacity


In [109]:
m1=[] 
for i in range(25):
  m1.append(model1(np.array(value_slot[slot[i]]),np.array(value_value[slot[i]]),12,50,len(value_value[slot[i]][0])))

Epoch 1/3
98/98 [==============================] - 15s 94ms/step - loss: 0.0483 - accuracy: 0.9722
Epoch 2/3
98/98 [==============================] - 9s 93ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/3
98/98 [==============================] - 9s 93ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 1/3
9/9 [==============================] - 4s 96ms/step - loss: 1.8323 - accuracy: 0.4816
Epoch 2/3
9/9 [==============================] - 1s 93ms/step - loss: 1.0567 - accuracy: 0.7323
Epoch 3/3
9/9 [==============================] - 1s 93ms/step - loss: 1.0427 - accuracy: 0.7343
Epoch 1/3
4/4 [==============================] - 5s 165ms/step - loss: 1.5826 - accuracy: 0.4930
Epoch 2/3
4/4 [==============================] - 0s 89ms/step - loss: 1.1173 - accuracy: 0.8375
Epoch 3/3
4/4 [==============================] - 0s 86ms/step - loss: 0.8937 - accuracy: 0.8167
Epoch 1/3
4/4 [==============================] - 5s 112ms/step - loss: 2.8632 - accuracy: 0.2439
Epoch 2/3
4/4 [========

1/1 [==============================] - 3s 3s/step - loss: 0.6971 - accuracy: 0.6667
Epoch 2/3
1/1 [==============================] - 0s 92ms/step - loss: 0.3793 - accuracy: 1.0000
Epoch 3/3
1/1 [==============================] - 0s 88ms/step - loss: 0.1971 - accuracy: 1.0000
Epoch 1/3
1/1 [==============================] - 3s 3s/step - loss: 0.7352 - accuracy: 0.0000e+00
Epoch 2/3
1/1 [==============================] - 0s 31ms/step - loss: 0.6599 - accuracy: 1.0000
Epoch 3/3
1/1 [==============================] - 0s 30ms/step - loss: 0.5002 - accuracy: 1.0000


In [110]:
m2=model1(np.array(inp_slot),np.array(out_slot),12,50,25)

Epoch 1/3
151/151 [==============================] - 35s 209ms/step - loss: 1.5523 - accuracy: 0.6327
Epoch 2/3
151/151 [==============================] - 31s 207ms/step - loss: 0.9155 - accuracy: 0.7263
Epoch 3/3
151/151 [==============================] - 31s 202ms/step - loss: 0.7456 - accuracy: 0.7773


In [111]:
m2.evaluate(np.array(user_emb[2940:]),np.array(slot_arr[2940:]))

40/40 [==============================] - 3s 70ms/step - loss: 0.7489 - accuracy: 0.8715


[0.7488791346549988, 0.8715305328369141]

In [112]:
m=[]
for i in range(12):  
  m.append(model1(np.array(inp[i][:2940]),np.array(first_arr[i][:2940]),len(inp[i][0]),50,cnt[i])) 

Epoch 1/3
92/92 [==============================] - 25s 231ms/step - loss: 3.6801 - accuracy: 0.2623
Epoch 2/3
92/92 [==============================] - 21s 229ms/step - loss: 2.4933 - accuracy: 0.4093
Epoch 3/3
92/92 [==============================] - 21s 229ms/step - loss: 2.1863 - accuracy: 0.4624
Epoch 1/3
92/92 [==============================] - 27s 255ms/step - loss: 3.8328 - accuracy: 0.2398
Epoch 2/3
92/92 [==============================] - 23s 254ms/step - loss: 2.2419 - accuracy: 0.4867
Epoch 3/3
92/92 [==============================] - 24s 256ms/step - loss: 1.7401 - accuracy: 0.5985
Epoch 1/3
92/92 [==============================] - 30s 289ms/step - loss: 4.7882 - accuracy: 0.0947
Epoch 2/3
92/92 [==============================] - 26s 283ms/step - loss: 3.2573 - accuracy: 0.3043
Epoch 3/3
92/92 [==============================] - 26s 284ms/step - loss: 2.5388 - accuracy: 0.4420
Epoch 1/3
92/92 [==============================] - 32s 316ms/step - loss: 4.7320 - accuracy: 0.1021


In [113]:
for i in range(12):
  print(m[i].evaluate(np.array(inp[i][2940:]),np.array(first_arr[i][2940:]),50,cnt[i]))

[2.3898913860321045, 0.4528152346611023]
[1.776123285293579, 0.6161776185035706]
[2.5638768672943115, 0.5099127888679504]
[2.6320109367370605, 0.5114988088607788]
[2.5699360370635986, 0.4726407527923584]
[2.6849944591522217, 0.4916732609272003]
[2.4465842247009277, 0.5432196855545044]
[2.409608840942383, 0.5305313467979431]
[2.07124924659729, 0.6058683395385742]
[1.8493220806121826, 0.6471054553985596]
[1.5489704608917236, 0.6867565512657166]
[1.3841608762741089, 0.7351308465003967]


In [114]:
import numpy as np
def select(slt,user_uttr,slct,test_i):
  if 1==0:
    return slct 
  else:
    slt_arr=get_pos(slt) 
    inp_to_db=[] 
    for i in range(12):
      inp_to_db.append([0]*50) 
  for slt_i in slt_arr:
    if slot[slt_i] in key_to_pos:
      val_arr=m1[slt_i].predict(np.array([user_uttr])) 
      ind=np.argmax(val_arr[0]) 
      ans=values_of_slot[slot[slt_i]][ind]
      for i in range(len(user_uttr)):
        if np.argmax(copy_model[slot[slt_i]].predict(np.array([user_uttr[i]]))[0])==1:
          if user_arr[test_i][i] in word_to_emb:
            ans=user_arr[test_i][i] 
          break
      inp_to_db[key_to_pos[slot[slt_i]]]=word_to_emb[ans] 
      print(slot[slt_i],ans)
  mx=0
  db_index=1000 
  for each_product in db:
    temp=0
    for slt_i in slt_arr:
      if slot[slt_i] in slot_to_key and slot[slt_i] in key_to_pos and each_product[slot_to_key[slot[slt_i]]]==ans:
        temp+=1 
    if temp>mx:
      db_index=db.index(each_product)
  return db_index
    

In [115]:
count=0
slct=0
out=whole_beam_search(user_emb[2944:],m,first) 
slot_out=m2.predict(np.array(user_emb[2944:])) 


In [116]:
slot_to_key['cost']='cost_price'
slot_to_key['radio']='rad_bin'
for i in range(2944,len(User_Arr)): 
  print(User_Arr[i])
  slct=select(slot_out[i-2944],user_emb[i],slct,i) 
  sent=out[i-2944] 
  for k in db[slct]:
    sent=sent.replace(k,str(db[slct][k]))  
  count+=bleu_score(sent,Agent_Arr[i])
  print(sent)
  print('') 
print(count/(len(User_Arr)-2944))

thanks a ton


c:\users\saswat\appdata\local\programs\python\python37\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\users\saswat\appdata\local\programs\python\python37\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\users\saswat\appdata\local\programs\python\python37\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lowe

my pleasure . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

hey , i want to buy a phone  . 
great choice , i am booking it for you . <pad> <pad>

i want a 2017 released year phone under 350 euro
cost 350
year 2017
le s3 by leeco is a good good phone . <pad> <pad> <pad>

what is its camera quality ?
it is 16 mp <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

that is too less . i want 13 mp for both primary and secondary camera
camera 13
le s3 by leeco is very good good phone <pad> <pad> <pad> <pad>

i would still prefer high mp phone  . 
le s3 by leeco is a good good phone . <pad> <pad> <pad>

what is its cost ?
it is 200 euro <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

okay , i will take this phone  . 
great choice , i am booking it for you . <pad> <pad>

you are a very good salesman  .  thanks a lot
my pleasure <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

i am looking for a good phone
sure , any specific features you have looking ? <pad> <pad> <pad>

i w

camera good
battery good
le s3 by leeco is a good good phone . <pad> <pad> <pad>

what is the price ?
it is 200 euro <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

is it new released phone ?
yes , i am booking it for you . <pad> <pad> <pad>

oh i want 2017 released phone
year 2017
brand samsung
radio yes
this phone by zte is very good phone . <pad> <pad> <pad>

i believe latest is the greatest so would go for new released phone
cost less
it has leeco is very good good phone . <pad> <pad> <pad>

what is the cost ?
it is 200 euro <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

ok can you book it for me
yes done <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

thanks for help
welcome and thank you so much for shopping with us . <pad>

hey , i need phone info  . 
great choice , i am booking it for you . <pad> <pad>

i want to buy a recent released gionee phone with good memory backup  . 
battery good
brand gionee
elife s5.5 by gionee is a good good phone . <pad> <pad> <pad>


brand micromax
w900 by micromax is a good good phone . <pad> <pad> <pad>

what about its weight ?
it is 153 gms <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

oh that 's too heavy  .  i want light weight phone
battery good
weight light
this phone is is very good good phone <pad> <pad> <pad> <pad>

my hand hurts carrying heavy weight phone
le s3 by leeco is a good good phone . <pad> <pad> <pad>

cool , what is its memory ?
it is 3000.0 gms <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

what is its cost ?
it is 200 euro <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

cool , i will purchase it  . 
great choice , i am booking it for you . <pad> <pad>

thanks a lot
welcome and thank you so much for shopping with us . <pad>

hi , can you please help me in selecting a phone ?
sure sure , i know what what kind ? <pad> <pad> <pad>

i want to buy a huawei phone with less weight
battery good
brand huawei
ideos s7 by huawei is a good good phone . <pad> <pad> <pad>

what is its display size
it

...

camera good
brand samsung
i617 blackjack ii by samsung is a good good phone . <pad> <pad> <pad>

what is its camera quality ?
it is 16 mp <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

what is its ram storage ?
it is 3000.0 mah <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

i want a samsung phone with more ram space
battery good
le s3 by leeco is a good good phone . <pad> <pad> <pad>

what is its cost ?
it is 200 euro <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

okay , i will take this phone  . 
great choice , i am booking it for you . <pad> <pad>

you are a very good salesman  .  thanks a lot
my pleasure <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

i want to purchase a nice phone
le s3 by leeco is a good good phone . <pad> <pad> <pad>

it should be recent released vivo phone
year 2017
brand vivo
radio yes
v880e by zte is a good good phone . <pad> <pad> <pad>

and yes os should be android
os and
racer by zte is a good good phone . <pad> <pad> <pad>

what is its 

os and
racer by zte is a good good phone . <pad> <pad> <pad>

is it a multisim phone ?
yes , i am booking it for you . <pad> <pad> <pad>

can you recommend multisim , android os and big screen size phone ?
color black
os and
racer by zte is a good good phone . <pad> <pad> <pad>

but i use three sims so would need multisim phone
le s3 by leeco is a good good phone . <pad> <pad> <pad>

what is its price ?
it is 200 euro <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

lovely it is under my budget  .  i will buy it  . 
cost less
sure , i am booking this for you . <pad> <pad> <pad>

thank you so much
you are most welcome <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

good evening , i need a phone  . 
sure , please tell me your model no ? <pad> <pad> <pad>

it should be black color apple phone
color black
os and
brand apple
iphone 3g by apple is a good good phone . <pad> <pad> <pad>

what is its ram ?
it is 3000.0 mah <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

what is its price ?
it

camera good
battery good
cost 200
le s3 by leeco is a good good phone . <pad> <pad> <pad>

no thats all  . 
okay , i am booking this for you . <pad> <pad> <pad>

what is its cost
it is 200 euro <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

what is its released yr ?
it 's in gray <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

oh , but i want newly launched 2017 mobile
camera good
year 2017
radio yes
v880e by zte is a good good phone . <pad> <pad> <pad>

it sounds great  . 
yes , i am booking this for you . <pad> <pad> <pad>

ok book this for me
yes done <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

thanks for assistance  . 
welcome and thank you so much for shopping with us . <pad>

hello , show me a trending phone
sure , any specific features you are looking ? <pad> <pad> <pad>

i need to buy samsung phone that has good battery life at less cost  . 
battery good
brand samsung
i617 blackjack ii by samsung is a good good phone . <pad> <pad> <pad>

how much is its pric

cost less
year latest
radio yes
this phone is is very good good phone . <pad> <pad> <pad>

yeah but i want latest phone
great choice , i am booking it for you . <pad> <pad>

what is its price ?
it is 200 euro <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

ok can you book it for me
yes done <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

thanks a lot for your time
welcome and thank you so much for choosing us <pad> <pad> <pad>

hello , i want a new phone  . 
great choice , i am booking it for you . <pad> <pad>

i want google phone with 64 gb internal ram
le s3 by leeco is a good good phone . <pad> <pad> <pad>

what is its display size ?
it is 3000.0 pixel <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

oh i ca n't buy this  .  it 's big  .  i want medium display size that 's good enough to do anything on phone  . 
cost less
this phone is is very good good phone . <pad> <pad> <pad>

i just want my regular fit size phone that can fit in my pocket
le s3 by leeco is a good g

os sym
this phone is you know what looking kind ? <pad> <pad> <pad>

what is its color ?
it 's in gray <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

okay , i will book this phone
great choice , i am booking it for you . <pad> <pad>

thanks a lot  . 
my pleasure . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

hello , i want to purchase a phone which has been reviewed by many
cost less
le s3 by leeco is a good good phone . <pad> <pad> <pad>

i want a phone in which both gps and radio works
camera good
radio yes
v880e by zte is a good good phone . <pad> <pad> <pad>

what is its internal ram storage ?
it is 3000.0 mah <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

i want a phone with more internal ram space
le s3 by leeco is a good good phone . <pad> <pad> <pad>

but phone 's memory plays an important role in phone 's performance
camera good
le s3 by leeco is a good good phone . <pad> <pad> <pad>

what is its cost ?
it is 200 euro <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

In [117]:
count=0
slct=0
out=whole_beam_search(user_emb[:2944],m,first) 
slot_out=m2.predict(np.array(user_emb[:2944])) 
for i in range(2944): 
  print(User_Arr[i])
  sent=out[i]  
  count+=bleu_score(sent,Agent_Arr[i])
  print(sent)
  print('') 
print(count/2944)

hi , can you help me in buying a phone ?
sure , any specific model you are looking ? <pad> <pad> <pad>

it should have good camera and battery capacity
model_name by brand_name is a good good phone . <pad> <pad> <pad>

how much does it cost ?
it is cost_price money_unit <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

what is it 's color ?
the color is color_name <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

oh , is it possible in black ?
this phone is is very good good phone . <pad> <pad> <pad>

but i like black
you can should welcome <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

what is it 's cost ?
it is cost_price money_unit <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

ok can you book redmi a for me
yes done <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

thanks for help
welcome and thank you so much for shopping with us . <pad>

hi , i want to buy a new phone
sure , any specific model you are looking ? <pad> <pad> <pad>

it should have os android and dual si

model_name by brand_name is a good good phone . <pad> <pad> <pad>

what are its specification ?
it is weight_val mp <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

is it available in android ?
it 's color_name mp <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

but i want android phone only  . 
model_name by brand_name is a good good phone . <pad> <pad> <pad>

ok but i have been using android phone for a long time  . 
model_name by brand_name is a good good phone . <pad> <pad> <pad>

it looks good  . 
sure , i am booking this for you . <pad> <pad> <pad>

ok book this one for me
yes done <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

thanks for helping me  . 
welcome and thank you so much for shopping with us . <pad>

hello , can you help me in selecting a phone ?
sure sure , i know what are your ? <pad> <pad> <pad>

i want to buy a blue color dualsim phone for my son  . 
sure , model_name by brand_name is good phone . <pad> <pad> <pad>

i have budget constraints , it sho

it is cam_val mp <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

oh it is too less , i want at least 20 mp primary camera
this phone is is very good good phone <pad> <pad> <pad> <pad>

still i would go for higher mp phone  .  show me a phone in other brand
model_name by brand_name is a good good phone . <pad> <pad> <pad>

cool , what is its memory ?
it is bat_size gms <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

what is its cost ?
it is cost_price money_unit <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

cool , i will purchase it  . 
great choice , i am booking it for you . <pad> <pad>

thanks a lot
welcome and thank you so much for shopping with us . <pad>

good evening , i am looking for best smartphone
sure , please tell me your model no <pad> <pad> <pad> <pad>

nothing in particular , i want a 2017 released android phone which is heavily in demand
model_name by brand_name is a good good phone <pad> <pad> <pad> <pad>

what is its color ?
it 's in color_name <pad> <pad> <pad> 

it is bat_size pixel <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

oh , but i want a phone which has small display size  . 
model_name by brand_name is very good good phone <pad> <pad> <pad> <pad>

but still i am not comfortable with big display size phone
model_name by brand_name is very good good phone . <pad> <pad> <pad>

what is its price ?
it is cost_price money_unit <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

perfect , i will take it  .  kindly gift wrap it  . 
sure , i am booking this for you . <pad> <pad> <pad>

thanks a ton
my pleasure . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

i am looking for a good phone
sure , any specific features you have looking ? <pad> <pad> <pad>

i want a small display size phone with good memory
model_name by brand_name is a good good phone . <pad> <pad> <pad>

i want in black color  . 
sure , you am booking this for you . <pad> <pad> <pad>

what is its os
it 's cam_val <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

oh 

it is weight_val mah <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

oh , that is too less , i want a phone of 3 gb ram and 32 gb internal ram
this phone is is very good good phone <pad> <pad> <pad> <pad>

but still i would opt for higher ram
model_name by brand_name is a good good phone . <pad> <pad> <pad>

what is its price ?
it is cost_price money_unit <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

perfect , i will take it  .  kindly gift wrap it  . 
sure , i am booking this for you . <pad> <pad> <pad>

thanks a ton
my pleasure . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

hi , i want to buy a phone for my daughter
sure , any specific model you are looking ? <pad> <pad> <pad>

yes , mobile should have good battery and internal storage  . 
model_name by brand_name is a good good phone . <pad> <pad> <pad>

what is its color ?
it 's in color_name <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

oh but i am looking for white color
you can buy model_name by brand_name <pad>

...


i want to buy a good processor phone with less weight and high battery life
model_name by brand_name is a good good phone . <pad> <pad> <pad>

how much does it cost ?
it is cost_price money_unit <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

does it uses android os ?
it 's color_name mp <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

i want mic os  . 
yes done <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

okay book this model for me  . 
yes done <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

thanks for helping me  . 
welcome and thank you so much for shopping with us . <pad>

hello , i want to buy a new phone  . 
sure , any specific model you are looking ? <pad> <pad> <pad>

i want to buy a 2017 released phone with radio facility
model_name by brand_name is a good good phone . <pad> <pad> <pad>

what is its battery ?
it is bat_size mah <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

what color is it ?
it 's in color_name color . <pad> <pad> <pad> <pad>

model_name by brand_name is very good good phone <pad> <pad> <pad> <pad>

but i am not a fan of big displays
model_name by brand_name is a good good phone . <pad> <pad> <pad>

how much does it cost ?
it is cost_price money_unit <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

okay book it for me  . 
great choice , i am booking it for you . <pad> <pad>

thanks for helping me  . 
welcome and thank you so much for shopping with us . <pad>

i need a phone
model_name by brand_name is a good good phone . <pad> <pad> <pad>

phone should have less weight and good primary camera
model_name by brand_name is a good good phone . <pad> <pad> <pad>

what is its color ?
it 's in color_name <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

whta is its primary camera ?
it is cam_val mp <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

oh that 's too less  .  i want at least 16 mp primary camera
this phone by brand_name is a good phone . <pad> <pad> <pad>

i would like to go for 16 mp or more camera phone

model_name by brand_name is a good good phone . <pad> <pad> <pad>

ok please complete the booking process  . 
it 's done . thank for buying . <pad> <pad> <pad> <pad>

thanks a lot for your time
welcome and thank you so much for choosing us <pad> <pad> <pad>

i need a phone which fulfills all my requirements  . 
model_name by brand_name is a good good phone . <pad> <pad> <pad>

i want to buy a oppo dual camera phone
model_name by brand_name is a good good phone . <pad> <pad> <pad>

what is its battery ?
it is bat_size mah <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

oh that 's very less  .  i need a long lasting battery phone
model_name by brand_name is very good good phone <pad> <pad> <pad> <pad>

i do lot of surfing so need high quality battery phone
model_name by brand_name is a good good phone . <pad> <pad> <pad>

ok please complete the booking process  . 
it 's done . thank for buying . <pad> <pad> <pad> <pad>

thanks a lot for your time
welcome and thank you so much for choosi

welcome and thank you so much for shopping with us . <pad>

hello , i want to buy a phone
sure , any specific model you are looking ? <pad> <pad> <pad>

yes , i need a gps operated phone with small display size
model_name by brand_name is a good good phone . <pad> <pad> <pad>

what is the price ?
it is cost_price money_unit <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

whats is its battery ?
it is bat_size mah <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

it is not enough  .  i want high battery
model_name by brand_name is very good good phone . <pad> <pad> <pad>

ok , can you please book it for me
yes done <pad> . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

thank you
you are most welcome <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

hey , i need phone info  . 
great choice , i am booking it for you . <pad> <pad>

it should be gionee company with less weight
model_name by brand_name is a good good phone . <pad> <pad> <pad>

its color should be white
model_name by brand_na

it is bat_size mah <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

fine , i will take it  . 
great choice , i am booking it for you . <pad> <pad>

thanks a ton
my pleasure . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

good evening , i want to buy a cell phone
sure , any specific model you are looking ? <pad> <pad> <pad>

i want to buy a micromax phone
model_name by brand_name is a good good phone . <pad> <pad> <pad>

what is its battery ?
it is bat_size mah <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

what is its cost ?
it is cost_price money_unit <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

sorry that 's little expensive for me  .  my budget is lttle less somewhere around 200 euro
sure , i am booking this for you . <pad> <pad> <pad>

i take your point but i do n't have that much budget
model_name by brand_name is a good good phone . <pad> <pad> <pad>

what is its os ?
it 's color_name <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

i will take it  . 
great c

In [118]:
count=0
slct=0
tot=0

slot_out=m2.predict(np.array(user_emb[:3000])) 
for i in range(3000): 
  if slot[np.argmax(slot_out[i])] in Slot_Arr[i]:
    count+=1 
  tot+=1

print(count/tot)

0.906


In [119]:
print(word_to_emb['samsung'])
print(word_to_emb['asus']) 
print(word_to_emb['nokia'])

[0.038318, -0.47761, 1.5479, 2.1977, -0.37663, -0.3043, 0.43616, -0.88477, -0.16857, 0.65115, 0.79176, -0.2107, -0.7319, -0.4491, -0.1263, 0.68891, -1.0975, 1.1121, -0.0386, -0.42534, 0.47778, -0.51349, -0.35801, -0.81789, -0.8123, -0.78331, -0.27652, -0.27396, -1.3298, -0.96249, 1.2821, 0.0098853, 0.57195, 0.72032, -0.0028576, -0.52664, -0.23956, -0.055702, -0.26017, -1.5175, 0.20478, -0.50222, -0.2321, -0.0041807, 1.5633, -0.79419, 0.34273, 0.25563, -0.64646, -0.45282]
[0.1214, -1.6385, 1.4713, 0.011653, -0.60636, -0.26422, -0.2934, -1.7427, 0.083684, 0.58321, 0.48483, 0.34426, 0.34291, 0.73514, -0.23917, -0.11382, -1.6707, 1.0467, 0.45641, -0.27456, -0.19535, -1.3659, 0.092267, -0.023367, 0.21419, 0.30424, -0.66461, -0.32736, -1.0596, -0.033881, -0.99766, 0.18998, -0.51495, 0.275, 0.11761, 0.12837, 0.13345, -0.12528, 0.17638, -0.049267, -0.5132, 0.06683, -1.215, -0.19476, 0.35268, 0.77722, 0.24524, 0.050067, 0.675, -0.15137]
[0.7177, -0.3894, 1.4676, 1.5313, -0.59071, -0.44679, -0.3

In [120]:
first_emb_inp=[]
for j in range(12):
  first_emb_inp.append([])
  for i in range(4201):
    first_emb_inp[j].append([first_emb[j][i]])

In [121]:
temp_mp={}
for arr in agent_arr:
  for word in arr:
    if word not in temp_mp:
      temp_mp[word]=0 
for word in temp_mp:
  print(word) 

hey
,
sure
what
kind
of
phone
do
you
need
?
model_name
by
brand_name
is
one
the
best
for
camera
as
its
cost
money_unit
cost_price
only
<pad>
color
oppo
aa
color_name
sir
this
model_nameas
strong
features
but
available
in
white
.
can
take
it
's
yes
done
i
am
booking
welcome
and
thank
so
much
shopping
with
us
hello
want
suited
to
your
model
gold
trending
these
go
model_namey
costs
ref
no
ref_no
visit
again
hi
sort
are
looking
moto
suits
requirements
has
dual
sim
release
year
year_name
os_name
response
very
good
mm
os
getting
review
buyers
liking
okay
then
a
option
type
purchase
meets
picture
quality
cam_val
rating_score
user
rating
feature
there
on
even
though
costly
customers
prefering
also
we
have
choosing
would
be
bat_size
mah
battery
will
last
long
came
give
model_name0
percent
discount
problem
buy
please
specify
attributes
been
reviewed
over
rating_scorerating_score000
users
memory
mem_size
gb
enough
daily
use
rs
rating_score800
people
my
pleasure
pack
reference
morning
basic
needs


In [122]:
def similarity_match(arr1,arr2):
    sm=0
    for i in range(12):
        if arr1[i]!=[0]*50:
            for j in range(50):
                sm+=abs(float(arr1[i][j])*100-float(arr2[i][j])*100) 
    return sm 
    
temp_db=[]
temp_db.append([0]*50)
temp_db.append([0]*50)
temp_db.append([0]*50)
temp_db.append(word_to_emb['6']) 
temp_db.append([0]*50)
temp_db.append([0]*50)
temp_db.append([0]*50)
temp_db.append([0]*50)
temp_db.append(word_to_emb['samsung']) 
temp_db.append([0]*50) 
temp_db.append(word_to_emb['200']) 
temp_db.append([0]*50)
match=100000000
pos=0
for i in range(len(db_in)):
    temp_match=similarity_match(temp_db,db_in[2183])
    if temp_match<=match:
        match=temp_match 
        pos=i 
print(db[pos])
sm=0
for i in range(50):
    sm+=abs(float(word_to_emb['samsung'][i])-float(word_to_emb['samsung'][i]))
print(sm)   

{'ref_no': '2516', 'cost_price': 180, 'money_unit': 'euro', 'month_name': 'july', 'year_name': 2010, 'review_person': '3500', 'rating_score': '4.4', 'cam_val': 3.15, 'mem_size': 0.001953125, 'bat_size': 1100.0, 'color_name': 'black', 'os_name': 'and', 'model_name': 'racer', 'brand_name': 'zte', 'rad_bin': 'no', 'size_val': 143, 'weight_val': 100}
0.0


In [ ]:
out=m1.predict(np.array(user_emb[2940:])) 
out_slot=m2.predict(np.array(user_emb[2940:])) 
out_val=m2.predict(np.array(user_emb[2940:])) 
n=0
for i in range(200,1000):
  q=(User_Arr[i+2940])
  act=ug_action[pos(out[i])]
  a=action_to_response[act]
  slt=slot[pos(out_slot[i])] 
  val=value[pos(out_val[i])]
  arr=gen_proper_response(q,a,act,slt,val,n) 
  a=arr[0] 
  n=arr[1] 
  bleu_score(Agent_Arr[i+2940],a) 
  print(q,a,sep='\n')
  print('')

In [ ]:
ott=m_temp.predict(np.array(inp_span[:2000]))
for o in ott:
  print(db[np.argmax(o)]['brand_name'])

In [ ]:
import random 
def gen_proper_response(q,res,act,slt,val,n): 
  if act=='feedback' or act=='confirm':
    if slt!='null':
      res=slot_to_response[slt] 
    else:
      res=action_to_response['confirm'] 
  for b in Brand_Arr:
    if b in q:
      res=action_to_response['confirm'] 
      val=b
      break
  if 'model_name' in res and 'brand_name' in res:
    if val in Brand_Arr:
      for i in range(len(db)):
        phone=db[i]
        if phone['brand_name']==val:
          n=i 
          break
    elif val in os_arr:
      for i in range(len(db)):
        phone=db[i] 
        if phone['os_name']==val:
          n=i 
          break 
    else:
      n=random.randint(0,2000) 
  for k in db[n].keys():
    if k in res:
      res=res.replace(k,str(db[n][k])) 
  return (res,n)

In [ ]:
action_to_response={} 
action_to_responses={} 
for u in U_Action_Arr:
  if u not in action_to_responses:
    action_to_responses[u]={}
    action_to_response[u]=''
for i in range(4201):
  if Agent_Arr[i] not in action_to_responses[U_Action_Arr[i]]:
    action_to_responses[U_Action_Arr[i]][Agent_Arr[i]]=1 
  else:
    action_to_responses[U_Action_Arr[i]][Agent_Arr[i]]+=1 
for u in action_to_response:
  mx=0 
  st='' 
  for reply in action_to_responses[u]:
    if action_to_responses[u][reply]>mx:
      mx=action_to_responses[u][reply] 
      st=reply 
  action_to_response[u]=st    
action_to_response['rec_slt']='model_name by brand_name is a good phone for you.' 
action_to_response['confirm']='model_name by brand_name is a good phone for you.' 

In [ ]:
slot_to_response={} 
slot_to_responses={} 
for u in Slot_Arr:
  if u not in slot_to_responses:
    slot_to_responses[u]={}
    slot_to_response[u]=''
for i in range(4201):
  if Agent_Arr[i] not in slot_to_responses[Slot_Arr[i]]:
    slot_to_responses[Slot_Arr[i]][Agent_Arr[i]]=1 
  else:
    slot_to_responses[Slot_Arr[i]][Agent_Arr[i]]+=1 
for u in slot_to_response:
  mx=0 
  st='' 
  for reply in slot_to_responses[u]:
    if slot_to_responses[u][reply]>mx:
      mx=slot_to_responses[u][reply] 
      st=reply 
  slot_to_response[u]=st    
slot_to_response={'null': 'my pleasure', 'camera': 'its picture quality is very good . it has cam_val mp of camera', 'color': 'this model_nameas strong features but available in color_name only .', 'os': 'it has os_name operating system in it.', 'touch': 'sure , model_name by brand_name will be a good option for you . it is a sturdy phone also .', 'cost': 'cost_price money_unit is its price', 'memory': 'mem_val is good enough for daily use', 'price': 'model_name has a price of cost_val.', 'year': 'it was released in year_name', 'company': 'model_name by brand_name is available to us.', 'review': 'it is getting good response . it has recently come in the market so less have reviewed it . many customers also forgot to give the feedback . trust me you will not regret buying it', 'dual': 'sure , we have many dual sim mobiles in our store . is there any other specific features you are looking for ?', 'rating': 'it has been rated rating_val by review_person.', 'battery': 'bat_val mah is good enough to lasts for a day .', 'weight': 'its weight is weight_val. ', 'brand': 'model_name by brand_name will be good for you.', 'radio': 'rad_bin it has rad_bin radio', 'sim': 'yes it has dual sim .', 'ram': 'it have mem_size gb of memory .', 'size': 'size_val is its size', 'gps': 'model_name by brand_name meets your requirements .', 'nll': 'yes it works .', 'model': 'mode_name is its name', 'apple': 'model_name by brand_name released date is year_name'}

In [ ]:
first_mp=[]
num_to_first=[] 
first_arr_mp=[]
first_arr=[]
count=[]
for j in range(12):
  first_mp.append({}) 
  num_to_first.append([])
  first_arr_mp.append({})
  cnt=0  
  for i in range(4201):
    if agent_arr[i][j] not in first_mp[j]:
      first_mp[j][agent_arr[i][j]]=cnt 
      num_to_first[j].append(agent_arr[i][j])
      cnt+=1
  count.append(cnt) 
  for k in first_mp[j]:
    first_arr_mp[j][k]=[]
    for i in range(cnt):
      if i==first_mp[j][k]:
        first_arr_mp[j][k].append(1)
      else: 
        first_arr_mp[j][k].append(0) 
  first_arr.append([]) 
  for i in range(4201):
    first_arr[j].append(first_arr_mp[j][agent_arr[i][j]])
  
print(count) 

In [ ]:
def select(slots,values,pos):
  if slot[mp_sl['null']]==1:
    return pos 
  pos_arr=[]
  for i in range(len(slots)):
    if slots[i]>0.1:
      if slot[i]=='null':
        return pos
      key=slot_to_key[slot[i]] 
      for j in len(db):
        if db[j][key].lower()==values[value[i]]:
          pos_arr.append(j) 
      if values[value[i]] in ['high','good','expensive','more']:
        mx=0 
        mx_pos=0
        for j in len(db):
          if isfloat(db[j][key])==1 and float(db[j][key])>mx:
            mx=float(db[j][key]) 
            mx_pos=j 
        pos_arr.append(mx_pos) 
      if values[value[i]] in ['less','low','cheap','bad']:
        mx=10000000 
        mx_pos=0
        for j in len(db):
          if isfloat(db[j][key])==1 and float(db[j][key])<mx:
            mx=float(db[j][key]) 
            mx_pos=j 
        pos_arr.append(mx_pos) 
  temp_mp={} 
  for p in pos:
    if p not in temp_mp:
      temp_mp[p]=1 
    else:
      temp_mp[p]+=1
  pos=0 
  mx=0
  for i in temp_mp:
    if temp_mp[i]>mx:
      mx=temp_mp[i] 
      pos=i 
  return pos 